In [4]:
import os
import PyPDF2
import re
import pandas as pd
from PyPDF2 import PdfReader 
import PyPDF2
from PyPDF2 import PdfFileReader


In [ ]:
# Function to extract invoice details like Pro number, BOL number, and Total Charges
def extract_invoice_details(text):
    pro_number = re.search(r'\bPro:\s*(\d+)', text)
    bol = re.search(r'\bBOL#:\s*(\d+)', text)
    total_charges = re.search(r'Total \n\nCharges: \n\n(\$[\d,.]+)', text)
    
    pro_number = pro_number.group(1) if pro_number else None
    bol = bol.group(1) if bol else None
    total_charges = total_charges.group(1) if total_charges else None

    return pro_number, bol, total_charges

# Function to extract route details from the text
def extract_route(text):
    route_pattern = r'(\d+)?\s*\n\n?([PD])?\s*\n\n?(.*?)\s*\n\n?([\d.]+)\s*\n\n?(.*?)\s*\n\n?([A-Z]{2})\s*\n\n?(\d{5})'    
    route_matches = re.findall(route_pattern, text)
    return route_matches

# Function to process a given PDF, extract relevant details, and return them as a dataframe
def extract_data_from_pdf(pdf_path):
    # Read the PDF and combine the text from all its pages
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        all_page_texts = [page.extract_text() for page in reader.pages]
    
    combined_text = ''.join(all_page_texts)
    
    # Split the text on the basis of 'Pro:' to get individual invoices
    invoices = re.split(r'Pro:', combined_text)
    invoices = [invoices[0]] + ["Pro:" + invoice for invoice in invoices[1:]]
    # Clean the invoices by replacing double newline characters with a space
    cleaned_invoices = [invoice.replace('\n\n', ' ') for invoice in invoices]

    # Extract specific details from each invoice and collect them in a list
    data = []
    for detail in cleaned_invoices:
        pro_number, bol, total_charges = extract_invoice_details(detail)
        route = extract_route(detail)
        for r in route:
            seq_num, type_, location, distance, city, state, zip_code = r
            data.append([pro_number, bol, total_charges, seq_num, type_, location, distance, city, state, zip_code])
    
    # Convert the list to a dataframe and return
    return pd.DataFrame(data, columns=["pro_number", "bol", "total_charges", "seq_num", "type_", "location", "distance", "city", "state", "zip_code"])

# Specify the directory path where the PDFs are stored
directory_path = "C:\\Users\\prapa001\\OneDrive - Corporate\\Documents\\Line haul PDF\\NFI\\INVOICES\\DAYVILLE\\"

# List all files in the directory
all_files = os.listdir(directory_path)
# Filter out only PDF files from the list
pdf_files = [file for file in all_files if file.endswith('.pdf')]

# For each PDF file, extract its data and store the dataframes in a list
dataframes = [extract_data_from_pdf(os.path.join(directory_path, pdf_file)) for pdf_file in pdf_files]

# Concatenate all the individual dataframes to create a master dataframe
master_df = pd.concat(dataframes, ignore_index=True)

# Display the head of the master dataframe for a quick check
print(master_df.head())


In [13]:
# Validating the extracted invoices at Pro: level 
#df_extracted.nunique()
# Filter the dataframe for the specific Pro number 11252093
df_specific_pro_example = master_df[master_df["pro_number"] == "11223187"]
df_specific_pro_example#.sum()

,pro_number,bol,total_charges,seq_num,type_,location,distance,city,state,zip_code
0,11223187,1677397,None,2,P,STAPLES DC,2.0,Putnam,CT,06260
1,11223187,1677397,None,3,D,3007L,179.0,Jersey City,NJ,07306
2,11223187,1677397,None,4,D,8422,3.0,Jersey City,NJ,07305
3,11223187,1677397,None,5,D,STAPLES DC,174.0,Putnam,CT,06260


In [9]:
master_df

,pro_number,bol,total_charges,seq_num,type_,location,distance,city,state,zip_code
0,11223187,1677397,None,2,P,STAPLES DC,2.0,Putnam,CT,06260
1,11223187,1677397,None,3,D,3007L,179.0,Jersey City,NJ,07306
2,11223187,1677397,None,4,D,8422,3.0,Jersey City,NJ,07305
3,11223187,1677397,None,5,D,STAPLES DC,174.0,Putnam,CT,06260
4,11223190,1677400,None,2,P,STAPLES FC,14.0,Plainfield,CT,06374
...,...,...,...,...,...,...,...,...,...,...
5175,11647988,1777291,None,4,D,DICOM COURIER,27.0,Hicksville,NY,11801
5176,11647988,1777291,None,5,D,STAPLES FC,160.0,Plainfield,CT,06374
5177,11675430,1784370,None,2,P,STAPLES FC,14.0,Plainfield,CT,06374
5178,11675430,1784370,None,3,D,3858C,43.0,East Hartford,CT,06108
